In [ ]:
import os
import torch
from torch import nn ,optim
from torch.utils.data import DataLoader,Dataset, TensorDataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose,transforms
import matplotlib.pyplot as plt
import pandas as pd
from torchvision.io import read_image
import numpy as np

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
print('Using {} device'.format(device))

In [ ]:
from pathlib import Path
import requests

DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

URL = "https://github.com/pytorch/tutorials/raw/master/_static/"
FILENAME = "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)

In [ ]:
import pickle
import gzip

with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
y_train

In [ ]:
x_train, y_train, x_valid, y_valid = map(
    torch.cuda.FloatTensor, (x_train, y_train, x_valid, y_valid))

In [ ]:
size = x_train.shape[0]
size2 = x_valid.shape[0]

In [ ]:
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(train_ds, batch_size=32)
test_ds = TensorDataset(x_valid, y_valid)
test_dl = DataLoader(test_ds, batch_size=32)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.conv5 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv6 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.conv7 = nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1)
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU(),
        )
    
    def forward(self, xb):
        xb = xb.view(-1, 1, 28, 28)
        xb = nn.functional.relu(self.conv1(xb))
        xb = nn.functional.relu(self.conv2(xb))
        xb = nn.functional.relu(self.conv3(xb))
        xb = nn.functional.relu(self.conv4(xb))
        xb = nn.functional.relu(self.conv5(xb))
        xb = nn.functional.relu(self.conv6(xb))
        xb = nn.functional.relu(self.conv7(xb))
        xb = nn.functional.avg_pool2d(xb, 4)
        xb = self.flatten(xb)
        xb = self.linear_relu_stack(xb)
        return xb.view(-1, xb.size(1))


In [ ]:
model = CNN().to(device)
dummy = torch.ones(64,1,28,28).to(device)
a = model(dummy)
a.shape

In [ ]:
lr = 0.009
model = CNN().to(device)
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.7)
softmax = nn.LogSoftmax(dim=1)
loss_fn = nn.NLLLoss()

https://stackoverflow.com/questions/59013109/runtimeerror-input-type-torch-floattensor-and-weight-type-torch-cuda-floatte/59013131

https://jbencook.com/pytorch-numpy-conversion/

https://neptune.ai/blog/pytorch-loss-functions

https://discuss.pytorch.org/t/runtimeerror-expected-object-of-scalar-type-long-but-got-scalar-type-float-when-using-crossentropyloss/30542

In [ ]:
for epoch in range(10):
    for batch, (xb, yb) in enumerate(train_dl):
        #print(len(xb))
        X, y = xb.to(device), yb.to(device)
        # Compute prediction error
        pred = model(X)
        #print(pred.shape,y.shape)
        loss = loss_fn(softmax(pred), y.long())
        loss.backward()
        with torch.no_grad():
            for p in model.parameters():
                p -= p.grad * lr
            model.zero_grad()
        if batch % 64 == 0 and epoch%2 ==0:
            loss, current = loss.item(), (batch)*len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
       

In [ ]:
test_loss, correct = 0, 0
print(size2)
for (xb, yb) in test_dl:
    X, y = xb.to(device), yb.to(device)
    pred = model(X)
    loss = loss_fn(softmax(pred), y.long())
    test_loss += loss.item()
    correct += (pred.argmax(1) == y).type(torch.float).sum().item()

test_loss /= size2
correct /= size2
print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [ ]:
torch.save(model, 'CNN.pth')